#**Importing Dataset from Kaggle**

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d vishalmane109/indian-currency-note-images-dataset-2020
! unzip indian-currency-note-images-dataset-2020.zip -d dataset

Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                         title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              215KB  2021-03-13 10:04:34            347         40  1.0              
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            367         53  1.0              
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44            295         45  1.0       

#**Using Image Generator for Data Augmentation and initializing paths for the datasets.**

Libraries used are:
```
tensorflow
matplotlib
```

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
train_path = "/content/dataset/Indian currency dataset v1/training"
test_path = "/content/dataset/Indian currency dataset v1/validation"

##Data Augmentation

In [ ]:
dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=5,
    horizontal_flip=True,
    vertical_flip=True,
)

train_dataset_aug = dataGen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

test_dataset_aug = dataGen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

Found 3566 images belonging to 8 classes.
Found 345 images belonging to 8 classes.


In [ ]:
 pretrained_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                                include_top=False,
                                                weights='imagenet')


In [ ]:
pretrained_model.trainable = False

In [ ]:
pretrained_model.summary()

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(pretrained_model)

In [ ]:
model.add(tf.keras.layers.GlobalAveragePooling2D())

In [ ]:
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(516, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam',
                loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(train_dataset_aug, validation_data=test_dataset_aug, epochs = 15)

Epoch 1/15
112/112 [==============================] - ETA: 0s - loss: 1.5383 - accuracy: 0.4720

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


112/112 [==============================] - 265s 2s/step - loss: 1.5348 - accuracy: 0.4732 - val_loss: 0.7566 - val_accuracy: 0.7246
Epoch 2/15
112/112 [==============================] - 259s 2s/step - loss: 0.6517 - accuracy: 0.7790 - val_loss: 0.5131 - val_accuracy: 0.8435
Epoch 3/15
112/112 [==============================] - 258s 2s/step - loss: 0.4465 - accuracy: 0.8533 - val_loss: 0.6459 - val_accuracy: 0.7681
Epoch 4/15
112/112 [==============================] - 254s 2s/step - loss: 0.3731 - accuracy: 0.8641 - val_loss: 0.4308 - val_accuracy: 0.8580
Epoch 5/15
112/112 [==============================] - 254s 2s/step - loss: 0.3051 - accuracy: 0.8979 - val_loss: 0.5071 - val_accuracy: 0.8493
Epoch 6/15
112/112 [==============================] - 255s 2s/step - loss: 0.2779 - accuracy: 0.9014 - val_loss: 0.3349 - val_accuracy: 0.8841
Epoch 7/15
112/112 [==============================] - 260s 2s/step - loss: 0.2338 - accuracy: 0.9260 - val_loss: 0.2823 - val_accuracy: 0.9014
Epoch 8/15

#**Plotting the graph of accuracy and loss against their validation values**

Using the plot function in ``` matplotlib.pyplot```, we can plot the graphs of accuracy vs validation_accuracy, as well as the graph of loss vs validation loss. 

We can also determine if our model is *overfitting* by analysing these graphs.

In [ ]:
plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend('lower right')

In [ ]:
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend('lower right')
